# Job Description Preprocessing and Extraction
This notebook is designed to call all functions from the Utils script, preprocess the *aufgaben* content, and extract sentence units and meaningful phrases.

In [6]:
import pandas as pd
from pathlib import Path
import config as C
from utils import (dedupe_lines, clean_text, split_units, extract_phrases, embed_texts, df_to_sqlite, as_json, get_spacy, find_span_in_text)

In [8]:
input_csv = Path(C.JOBS_INPUT_CSV)   #job_details file with the raw job descriptions
sqlite_out = Path(C.SQLITE_PATH)     #sqlite database where everything will be stored

print("Input CSV:", input_csv)
print("Output SQLite file:", sqlite_out)


Input CSV: /Users/denysheryavenko/Desktop/EL Thesis/data/job_details.csv
Output SQLite file: /Users/denysheryavenko/Desktop/EL Thesis/data/db.sqlite


In [10]:
#we check if there is an id column in the raw job dataset, if not, we create it, to have a primary key
def pick_or_create_job_id(df):
  
    for col in C.ID_CANDIDATES:
        if col in df.columns:
            return col

    df["job_id"] = df.reset_index().index + 1
    return "job_id"

### Load the raw data and inspect it

In [13]:
df = pd.read_csv(input_csv)
print("Total rows:", len(df))
print("Columns:", list(df.columns))

#check if exists:
if C.COL_TITLE not in df.columns:
    df[C.COL_TITLE] = ""

if C.COL_TEXT not in df.columns:
    raise ValueError("The text column '" + C.COL_TEXT + "' is missing.")

# identify or create a job ID
id_col = pick_or_create_job_id(df)

# summary
variables = []
dtypes = []
missing = []
not_missing = []

for col in df.columns:
    variables.append(col)
    dtypes.append(str(df[col].dtype))
    missing.append(df[col].isna().sum())
    not_missing.append(df[col].notna().sum())

schema_summary = pd.DataFrame({
    "variable": variables,
    "dtype": dtypes,
    "missing": missing,
    "non_missing": not_missing
})

print("Schema and missing values:")
display(schema_summary)



Total rows: 400
Columns: ['title', 'url', 'deadline', 'location', 'aufgaben', 'erfordernisse']
Schema and missing values:


,variable,dtype,missing,non_missing
0,title,object,0,400
1,url,object,0,400
2,deadline,object,0,400
3,location,object,0,400
4,aufgaben,object,25,375
5,erfordernisse,object,7,393
6,job_id,int64,0,400


### Clean and Dedupe Text

In [16]:
#### assisting viz to wrap the text when printing
from IPython.display import HTML

pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.width", 1000)

HTML("""
<style>
.output_area pre {
    white-space: pre-wrap;
}
</style>
""")
####

# dedupe the text
deduped_list = []
for text in df[C.COL_TEXT].fillna(""):
    deduped_list.append(dedupe_lines(text))
df["text_deduped"] = deduped_list

# simple cleaning with structure
clean_list = []
for text in df["text_deduped"]:
    clean_list.append(clean_text(text))
df["text_clean"] = clean_list

#show some examples
display(df[[id_col, C.COL_TEXT, "text_deduped", "text_clean"]].head(1))


,job_id,aufgaben,text_deduped,text_clean
0,1,"Die Verwendung als Vertragshochschullehrperson/Hochschullehrperson an der Privaten Pädagogischen Hochschule Burgenland erfolgt gem. § 48g des VBG bzw. § 200d BDG 1979.\nSie erfüllen Aufgaben und Tätigkeiten insbesondere in folgenden Bereichen:\n•\tLehre in Aus-, Fort- und Weiterbildung im Bereich der Primarstufe mit Fokus auf Sprachliche Bildung und Inklusive Pädagogik\n•\tBeratung von Studierenden, insbesondere Betreuung bei der Absolvierung der päda-gogisch-praktischen Studien und der Erstellung von Qualifizierungsarbeiten\n•\tMitarbeit im Zentrum für Minderheitenschulwesen, Mehrsprachigkeit und Inklusion\n•\tMitarbeit bei Aufgaben in der wissenschaftlich-berufsfeldbezogenen Forschung im ausgeschriebenen Bereich\n•\tMitwirkung an Organisations- und Verwaltungsaufgaben, einschließlich der\nEvaluierung und Qualitätssicherung im ausgeschriebenen Bereich\n•\tEntwicklung und Betreuung von Bildungsangeboten (Aus-, Fort- und Weiterbildung) im ausgeschriebenen Bereich\nDie Verwendung als Vertragshochschullehrperson/Hochschullehrperson an der Privaten Pädagogischen Hochschule Burgenland erfolgt gem. § 48g des VBG bzw. § 200d BDG 1979.\nSie erfüllen Aufgaben und Tätigkeiten insbesondere in folgenden Bereichen:\n•\tLehre in Aus-, Fort- und Weiterbildung im Bereich der Primarstufe mit Fokus auf Sprachliche Bildung und Inklusive Pädagogik\n•\tBeratung von Studierenden, insbesondere Betreuung bei der Absolvierung der päda-gogisch-praktischen Studien und der Erstellung von Qualifizierungsarbeiten\n•\tMitarbeit im Zentrum für Minderheitenschulwesen, Mehrsprachigkeit und Inklusion\n•\tMitarbeit bei Aufgaben in der wissenschaftlich-berufsfeldbezogenen Forschung im ausgeschriebenen Bereich\n•\tMitwirkung an Organisations- und Verwaltungsaufgaben, einschließlich der\nEvaluierung und Qualitätssicherung im ausgeschriebenen Bereich\n•\tEntwicklung und Betreuung von Bildungsangeboten (Aus-, Fort- und Weiterbildung) im ausgeschriebenen Bereich\nDie Verwendung als Vertragshochschullehrperson/Hochschullehrperson an der Privaten Pädagogischen Hochschule Burgenland erfolgt gem. § 48g des VBG bzw. § 200d BDG 1979.\nSie erfüllen Aufgaben und Tätigkeiten insbesondere in folgenden Bereichen:\n•\tLehre in Aus-, Fort- und Weiterbildung im Bereich der Primarstufe mit Fokus auf Sprachliche Bildung und Inklusive Pädagogik\n•\tBeratung von Studierenden, insbesondere Betreuung bei der Absolvierung der päda-gogisch-praktischen Studien und der Erstellung von Qualifizierungsarbeiten\n•\tMitarbeit im Zentrum für Minderheitenschulwesen, Mehrsprachigkeit und Inklusion\n•\tMitarbeit bei Aufgaben in der wissenschaftlich-berufsfeldbezogenen Forschung im ausgeschriebenen Bereich\n•\tMitwirkung an Organisations- und Verwaltungsaufgaben, einschließlich der\nEvaluierung und Qualitätssicherung im ausgeschriebenen Bereich\n•\tEntwicklung und Betreuung von Bildungsangeboten (Aus-, Fort- und Weiterbildung) im ausgeschriebenen Bereich","Die Verwendung als Vertragshochschullehrperson/Hochschullehrperson an der Privaten Pädagogischen Hochschule Burgenland erfolgt gem. § 48g des VBG bzw. § 200d BDG 1979.\nSie erfüllen Aufgaben und Tätigkeiten insbesondere in folgenden Bereichen:\n•\tLehre in Aus-, Fort- und Weiterbildung im Bereich der Primarstufe mit Fokus auf Sprachliche Bildung und Inklusive Pädagogik\n•\tBeratung von Studierenden, insbesondere Betreuung bei der Absolvierung der päda-gogisch-praktischen Studien und der Erstellung von Qualifizierungsarbeiten\n•\tMitarbeit im Zentrum für Minderheitenschulwesen, Mehrsprachigkeit und Inklusion\n•\tMitarbeit bei Aufgaben in der wissenschaftlich-berufsfeldbezogenen Forschung im ausgeschriebenen Bereich\n•\tMitwirkung an Organisations- und Verwaltungsaufgaben, einschließlich der\nEvaluierung und Qualitätssicherung im ausgeschriebenen Bereich\n•\tEntwicklung und Betreuung von Bildungsangeboten (Aus-, Fort- und Weiterbildung) im ausgeschriebenen Bereich","Die Verwendung als Vertragshochschullehr

### Save Cleaned Jobs to SQL

In [19]:
df_jobs_clean = df[[id_col, C.COL_TITLE, "text_deduped", "text_clean"]].copy()
df_to_sqlite(df_jobs_clean, sqlite_out, "jobs_clean")

print("Saved", len(df_jobs_clean), "rows to the table 'jobs_clean' in the file:", sqlite_out)


Saved 400 rows to the table 'jobs_clean' in the file: /Users/denysheryavenko/Desktop/EL Thesis/data/db.sqlite


### Split Clean Job Descriptions into Units

In [22]:
nlp = get_spacy()
rows = []

#we iterate through each row and split based on the rules from utils
for index, row in df.iterrows():
    text = row["text_clean"]

    units = split_units(text, nlp, min_words_per_unit=C.MIN_WORDS_PER_UNIT) #min words per unit avoids cases like und, z.B., etc. ...

    #we keep units 
    for u in units:
        if len(u) > 2: # and here we try to remove any leftover symbols like dashes, bullets that became units (just to be safe)...
        
            start, end = find_span_in_text(text, u, allow_fuzzy=False)
            rows.append((row[id_col], u, start, end))

print("Total units extracted:", len(rows))

# some statistics
if len(rows) > 0:

    job_unit_count = {} #number of units per job

    for job_id, unit, start, end in rows:
        if job_id not in job_unit_count:
            job_unit_count[job_id] = 0
        job_unit_count[job_id] += 1

    counts = list(job_unit_count.values())

    units_summary = pd.DataFrame({
        "": ["Jobs with units", "Min. units per job", "Max. units per job", "Avg. units per job"],
        "value": [
            len(job_unit_count),
            min(counts),
            max(counts),
            round(sum(counts)/len(counts), 2)
        ],
    })

    print("Unit summary per job:")
    display(units_summary)

/opt/anaconda3/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'de_core_news_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Total units extracted: 2798
Unit summary per job:


,,value
0,Jobs with units,375.00
1,Min. units per job,1.00
2,Max. units per job,45.00
3,Avg. units per job,7.46


### Embed Units and Load to SQL
The embedding process is done in smaller batches, because doing this all at once can crash the memory or be too slow.

In [25]:
parts = [] #this list holds the small dataframes before joining them to the main table

if len(rows) > 0:

    #go through the rows in small batches
    start_idx = 0
    while start_idx < len(rows):
        end_idx = start_idx + C.EMBED_BATCH_SIZE
        batch = rows[start_idx:end_idx]
        
        texts = []
        job_ids_for_batch = []
        unit_starts = []
        unit_ends = []

        for item in batch:
            job_id, unit_text, u_start, u_end = item
            job_ids_for_batch.append(job_id)
            texts.append(unit_text)
            unit_starts.append(u_start)
            unit_ends.append(u_end)

        #create the embeddings for this batch
        vectors = embed_texts(texts)

        #convert to json to store later
        embedding_json_list = []
        for v in vectors:
            embedding_json_list.append(as_json(v))

        #build a small data frame for this batch and add it to "parts"
        temp = pd.DataFrame({
            id_col: job_ids_for_batch,
            "unit_text": texts,
            "unit_start": unit_starts, #span start
            "unit_end": unit_ends, # span end
            "embedding_json": embedding_json_list
        })
        parts.append(temp)

        start_idx = end_idx  # move to the next batch

    #finally combine all batch data frames into one
    df_units = pd.concat(parts, ignore_index=True)

else:
    # in case there are no units at all (rare, but avoids errors)
    df_units = pd.DataFrame(columns=[id_col, "unit_text", "unit_start", "unit_end", "embedding_json"])

df_to_sqlite(df_units, sqlite_out, "job_units")

print("Saved", len(df_units))
print("First 5 embedded units:")
display(df_units.head(5))

Saved 2798
First 5 embedded units:


job_id                                                                                                                                                            unit_text  unit_start  unit_end                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

### Phrase Extraction

In [27]:
#extract the phrases
phrases = []

#iterate through each row in the data frame 
for i in range(len(df)):
    text = df.loc[i, "text_clean"]
    job_id_value = df.loc[i, id_col]

    phs = extract_phrases(
        text,
        max_candidates = C.KEYBERT_MAX_CANDIDATES,
        score_cutoff = C.KEYBERT_SCORE_CUTOFF
    )
    #save the phrase into a tuple
    for item in phs:
        surface = item["phrase_surface"]

        #we allow fuzzy matching here because the surface is normalized (stopwords/lemma/spacing)
        start, end = find_span_in_text(text, surface, allow_fuzzy=True, min_tokens_found=1)

        phrases.append((
            job_id_value,
            surface,
            item["phrase_lemma"],
            item["score"],
            item["pos_signature"],
            start,
            end
        ))

print("Total phrases extracted:", len(phrases))

Total phrases extracted: 7668


*Below are some statistics describing the extracted phrases.*

In [29]:
pos_info = {}

for i in range(len(phrases)): #we iterate through the tuple and count the occurrences of each POS tag
    job_id_value, surface, lemma, score, pos_sig, start, end = phrases[i]

    if pos_sig not in pos_info:
        pos_info[pos_sig] = {"count": 0, "sum_score": 0.0}
        
    pos_info[pos_sig]["count"] += 1
    pos_info[pos_sig]["sum_score"] += float(score)

pos_rows = [] #collect the counts and scores (+avg) and store them
for pos_sig in pos_info:
    count = pos_info[pos_sig]["count"]
    avg_score = pos_info[pos_sig]["sum_score"] / count
    pos_rows.append([pos_sig, count, avg_score])

pos_stats = pd.DataFrame(
    pos_rows,
    columns=["POS", "n_phrases", "avg. score"]
)

display(pos_stats)

,POS,n_phrases,avg. score
0,ADJ NOUN,2391,0.480627
1,VERB NOUN,770,0.474442
2,NOUN VERB,1926,0.482492
3,NOUN NOUN,2246,0.473721
4,NOUN ADJ,335,0.463773


In [31]:
phrase_counts = {}

for item in phrases:
    job_id_value = item[0]
    if job_id_value not in phrase_counts:
        phrase_counts[job_id_value] = 0
    phrase_counts[job_id_value] += 1

counts = []
for job_id_value in phrase_counts:
    counts.append(phrase_counts[job_id_value])

jobs_with_phrases = len(phrase_counts)
min_phr_per_job = min(counts)
max_phr_per_job = max(counts)
avg_phr_per_job = round(sum(counts)/len(counts),2)

phr_job_summary = pd.DataFrame({
    "": [
        "jobs_with_phrases",
        "min_phrases_per_job",
        "avg_phrases_per_job",
        "max_phrases_per_job",
    ],
    "value": [
        jobs_with_phrases,
        min_phr_per_job,
        avg_phr_per_job,
        max_phr_per_job,
    ],
})

print("Phrase count per job (distribution summary)")
display(phr_job_summary)


Phrase count per job (distribution summary)


,,value
0,jobs_with_phrases,371.00
1,min_phrases_per_job,1.00
2,avg_phrases_per_job,20.67
3,max_phrases_per_job,72.00


In [33]:
score_values = []
for item in phrases:
    score_values.append(float(item[3]))

min_score = min(score_values)
max_score = max(score_values)
avg_score = sum(score_values) / len(score_values)

score_summary = pd.DataFrame({
    "": ["min_score", "avg_score", "max_score"],
    "value": [min_score, avg_score, max_score],
})

print("KeyBERT score statistics (all extracted phrases)")
display(score_summary)

KeyBERT score statistics (all extracted phrases)


,,value
0,min_score,0.300000
1,avg_score,0.477715
2,max_score,0.883800


### Embed Phrases and Load to SQL
Here we embed in batches as well

In [36]:
parts = []

if len(phrases) > 0:
    start = 0 #same logic as for the unit embedding
    while start < len(phrases):
        end = start + C.EMBED_BATCH_SIZE
        batch = phrases[start:end]

        job_ids = []
        surfaces = []
        lemmas = []
        kb_scores = []
        pos_sigs = []
        texts = []
        phrase_starts = []
        phrase_ends = []

        for item in batch:
            job_id_value = item[0]
            surface = item[1]
            lemma = item[2]
            kb_score = item[3]
            pos_sig = item[4]
            p_start = item[5]
            p_end = item[6]

            job_ids.append(job_id_value)
            surfaces.append(surface)
            lemmas.append(lemma)
            kb_scores.append(kb_score)
            pos_sigs.append(pos_sig)
            texts.append(surface)   #we will embed the surface text because transformers are usually not trained on lemmatized versions, but on natural language
            phrase_starts.append(p_start)
            phrase_ends.append(p_end)

        vectors = embed_texts(texts)

        embedding_json_list = []
        for v in vectors:
            embedding_json_list.append(as_json(v))

        temp = pd.DataFrame({
            id_col: job_ids,
            "phrase_surface": surfaces,
            "phrase_lemma": lemmas,
            "KeyBert score": kb_scores,
            "pos_signature": pos_sigs,
            "phrase_start": phrase_starts,
            "phrase_end": phrase_ends,
            "embedding_json": embedding_json_list
        })
        parts.append(temp)
        start = end
        
    df_phrases = pd.concat(parts, ignore_index=True)

else:
    #in case no phrases extracted (just in case)
    df_phrases = pd.DataFrame(columns=[
        id_col,
        "phrase_surface",
        "phrase_lemma",
        "KeyBert score",
        "pos_signature",
        "phrase_start",
        "phrase_end",
        "embedding_json",
    ])


df_to_sqlite(df_phrases, sqlite_out, "job_phrases")
print("Saved", len(df_phrases))
print("Preview of phrase embeddings:")
display(df_phrases.head(5))


Saved 7668
Preview of phrase embeddings:


job_id                  phrase_surface                    phrase_lemma  KeyBert score pos_signature  phrase_start  phrase_end                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   